In [140]:
# Find-S Algorithm Implementation
def findS(df, target_column):
    # Seperate features and target
    features = df.drop(columns = target_column)
    target = df[target_column]
    # Declare S as None
    S = None
    # iterating each example in dataset
    for i, example in features.iterrows():
        if target.iloc[i]:
            # Initialize S as first positive example
            if i == 0:
                S = example.values
            # For rest positive example
            else:
                for j in range(len(S)):
                    # Generalize S where the attributes are not matched
                    if example.iloc[j] != S[j]:
                        S[j] = '?'
    return S

In [141]:
# Candidate Elimination Algorithm Implementation
def consistent(hypothesis, instance):
    # Check if a hypothesis is consistent with a positive instance
    for h, i in zip(hypothesis, instance):
        if h != "?" and h != i:
            return False
    return True

def more_general(h1, h2):
    # Check if hypothesis h1 is more general than h2
    for x, y in zip(h1, h2):
        if x != '?' and (x != y and y != '?'):
            return False
    return True

def more_specific(h1, h2):
    # Check if hypothesis h1 is more specific than h2
    return more_general(h2, h1)

def generalize_hypothesis(h, example):
    # Generalize a specific hypothesis minimally
    h = list(h)
    for i in range(len(h)):
        if h[i] is None:  # Initialize hypothesis
            h[i] = example.iloc[i]
        elif h[i] != example.iloc[i]:  # Generalize to '?'
            h[i] = '?'
    return tuple(h)

def specialize_hypothesis(h, example, features):
    # Specialize a general hypothesis minimally
    specializations = set()
    for i in range(len(h)):
        if h[i] == '?':
            for value in features.iloc[:, i].unique():
                if value != example.iloc[i]:
                    new_hypothesis = list(h)
                    new_hypothesis[i] = value
                    specializations.add(tuple(new_hypothesis))
    return specializations

def candidate_elimination(df, target_column):
    # Separate features and target
    features = df.drop(columns=[target_column])
    target = df[target_column]

    # Number of attributes
    num_attr = len(features.columns)

    # Initialize S (most specific boundary)
    S = {tuple([None] * num_attr)}

    # Initialize G (most general boundary)
    G = {tuple(['?'] * num_attr)}

    # Iterate through each example in the dataset
    for i, example in features.iterrows():
        # Positive Example
        if target.iloc[i]:
            # Remove hypotheses from G that are inconsistent with the example
            G = {g for g in G if consistent(g, example.values)}
            # Generalize S minimally to include the example
            S = {generalize_hypothesis(s, example) for s in S}
            # Remove from S any hypothesis which is more general than s
            S = {s for s in S if all(more_specific(s,other) for other in S)}

        # Negative Example
        else:
            # Remove hypotheses from S that are inconsistent with the example
            S = {s for s in S if not consistent(s, example.values)}
            # Specialize G minimally to exclude the example
            new_G = set()
            for g in G:
                if g == tuple(['?'] * num_attr) or (not consistent(g, example.values)):
                    new_G.update(specialize_hypothesis(g, example, features))
            # Remove hypotheses in G that are less specific than any hypothesis in S
            G = {g for g in new_G if any(more_general(g, s) for s in S)}
            # Remove hypotheses in G that are more specific than another hypothesis in G
            G = {g for g in G if all(more_general(g,other) for other in G)}

    return (S, G)

In [142]:
# Generating Version Space
from itertools import product
def generate_version_space(S, G):
    S = [s for s in S]
    G = [g for g in G]
    # Empty version space
    version_space = set()
    # For each specific and general boundary hypothesis
    for s in S:
        for g in G:
            # Including all domain values of attributes
            attr_domains = []
            for i in range(len(s)):
                if s[i] == g[i]:
                    attr_domains.append(s[i])
                elif s[i] == '?':
                    attr_domains.append(['?'])
                else:
                    attr_domains.append([s[i], '?'])
            # Extract nested lists and their indices
            nested_indices = [i for i, x in enumerate(attr_domains) if isinstance(x, list)]
            nested_lists = [attr_domains[i] for i in nested_indices]
            # Generate all possible combinations
            combinations = list(product(*nested_lists))
            for combo in combinations:
                temp_list = attr_domains.copy()
                for idx, value in zip(nested_indices, combo):
                    temp_list[idx] = value
                version_space.add(tuple(temp_list))
    return version_space

In [143]:
dict1 = dict()
dict1['Weather'] = ["Sunny", "Sunny", "Rainy", "Sunny"]
dict1['Temperature'] = ["Warm", "Warm", "Cold", "Warm"]
dict1['Humidity'] = ["Normal", "High", "High", "High"]
dict1['Wind'] = ["Strong", "Strong", "Strong", "Strong"]
dict1['Water'] = ["Warm", "Warm", "Warm", "Cold"]
dict1['Forecast'] = ["Same", "Same", "Change", "Change"]
dict1['EnjoySport'] = ["Yes", "Yes", "No", "Yes"]

In [144]:
import pandas as pd
df = pd.DataFrame(dict1)
print(df)
print()
print("After changing target variable to binary:")
df['EnjoySport'] = (df['EnjoySport']=='Yes').astype(int)
df

  Weather Temperature Humidity    Wind Water Forecast EnjoySport
0   Sunny        Warm   Normal  Strong  Warm     Same        Yes
1   Sunny        Warm     High  Strong  Warm     Same        Yes
2   Rainy        Cold     High  Strong  Warm   Change         No
3   Sunny        Warm     High  Strong  Cold   Change        Yes

After changing target variable to binary:


,Weather,Temperature,Humidity,Wind,Water,Forecast,EnjoySport
0,Sunny,Warm,Normal,Strong,Warm,Same,1
1,Sunny,Warm,High,Strong,Warm,Same,1
2,Rainy,Cold,High,Strong,Warm,Change,0
3,Sunny,Warm,High,Strong,Cold,Change,1


In [145]:
# Find-S Algorithm
specificHypothesis = findS(df, 'EnjoySport')
print("Specific Hyopthesis(S): ",specificHypothesis)

Specific Hyopthesis(S):  ['Sunny' 'Warm' '?' 'Strong' '?' '?']


In [146]:
# Candidate ELimination Algorithm
(S, G) = candidate_elimination(df, 'EnjoySport')

print("Specific Boundary: ", S)
print("General Boundary: ", G)

Version_Space = generate_version_space(S,G)
print("Version Space: ", Version_Space)

Specific Boundary:  {('Sunny', 'Warm', '?', 'Strong', '?', '?')}
General Boundary:  {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}
Version Space:  {('?', 'Warm', '?', 'Strong', '?', '?'), ('Sunny', '?', '?', '?', '?', '?'), ('Sunny', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', 'Strong', '?', '?'), ('?', 'Warm', '?', '?', '?', '?'), ('Sunny', 'Warm', '?', 'Strong', '?', '?')}
